In [1]:
import ast
import os
import shutil
import torch
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Image, clear_output
from sklearn.model_selection import StratifiedKFold, train_test_split
from tqdm import tqdm
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# !git clone https://github.com/WongKinYiu/ScaledYOLOv4.git
# !git clone https://github.com/thomasbrandon/mish-cuda
%cd /app/_data/ScaledYOLOv4/mish-cuda
!python setup.py build install
clear_output()
! git checkout yolov4-large
%cd /app/_data/ScaledYOLOv4

error: pathspec 'yolov4-large' did not match any file(s) known to git
/app/_data/ScaledYOLOv4


In [3]:
# customize iPython writefile so we can write variables

from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [4]:
if not os.path.exists("/app/_data/ScaledYOLOv4/data/DataFile"):
    os.mkdir("/app/_data/ScaledYOLOv4/data/DataFile")

In [5]:
%%writetemplate /app/_data/ScaledYOLOv4/data/DataFile/data.yaml
# train and val datasets (image directory or *.txt file with image paths)
train: /app/_data/yolo5_dataset/yolo5l6_skf_voi_1/images/train
val: /app/_data/yolo5_dataset/yolo5l6_skf_voi_1/images/val
# number of classes
nc: 1
# class names
names: ['opacity']

In [6]:
%%writetemplate /app/_data/ScaledYOLOv4/data/DataFile/customYOLOv4l.yaml
# parameters
nc: 1  # number of classes
depth_multiple: 1.0  # expand model depth
width_multiple: 1.25  # expand layer channels

# anchors
anchors:
  - [13,17,  22,25,  27,66,  55,41]  # P3/8
  - [57,88,  112,69,  69,177,  136,138]  # P4/16
  - [136,138,  287,114,  134,275,  268,248]  # P5/32
  - [268,248,  232,504,  445,416,  640,640]  # P6/64
  - [812,393,  477,808,  1070,908,  1408,1408]  # P7/128

# csp-p7 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1]],  # 0
   [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2
   [-1, 1, BottleneckCSP, [64]],
   [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 5-P3/8
   [-1, 15, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 7-P4/16
   [-1, 15, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]], # 9-P5/32
   [-1, 7, BottleneckCSP, [1024]],
   [-1, 1, Conv, [1024, 3, 2]], # 11-P6/64
   [-1, 7, BottleneckCSP, [1024]],
   [-1, 1, Conv, [1024, 3, 2]], # 13-P7/128
   [-1, 7, BottleneckCSP, [1024]],  # 14
  ]

# yolov4-p7 head
# na = len(anchors[0])
head:
  [[-1, 1, SPPCSP, [512]], # 15
   [-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [-6, 1, Conv, [512, 1, 1]], # route backbone P6
   [[-1, -2], 1, Concat, [1]],
   [-1, 3, BottleneckCSP2, [512]], # 20 
   [-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [-13, 1, Conv, [512, 1, 1]], # route backbone P5
   [[-1, -2], 1, Concat, [1]],
   [-1, 3, BottleneckCSP2, [512]], # 25
   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [-20, 1, Conv, [256, 1, 1]], # route backbone P4
   [[-1, -2], 1, Concat, [1]],
   [-1, 3, BottleneckCSP2, [256]], # 30
   [-1, 1, Conv, [128, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [-27, 1, Conv, [128, 1, 1]], # route backbone P3
   [[-1, -2], 1, Concat, [1]],
   [-1, 3, BottleneckCSP2, [128]], # 35
   [-1, 1, Conv, [256, 3, 1]],
   [-2, 1, Conv, [256, 3, 2]],
   [[-1, 30], 1, Concat, [1]],  # cat
   [-1, 3, BottleneckCSP2, [256]], # 39
   [-1, 1, Conv, [512, 3, 1]],
   [-2, 1, Conv, [512, 3, 2]],
   [[-1, 25], 1, Concat, [1]],  # cat
   [-1, 3, BottleneckCSP2, [512]], # 43
   [-1, 1, Conv, [1024, 3, 1]],
   [-2, 1, Conv, [512, 3, 2]],
   [[-1, 20], 1, Concat, [1]],  # cat
   [-1, 3, BottleneckCSP2, [512]], # 47
   [-1, 1, Conv, [1024, 3, 1]],
   [-2, 1, Conv, [512, 3, 2]],
   [[-1, 15], 1, Concat, [1]],  # cat
   [-1, 3, BottleneckCSP2, [512]], # 51
   [-1, 1, Conv, [1024, 3, 1]],

   [[36,40,44,48,52], 1, Detect, [nc, anchors]],   # Detect(P3, P4, P5, P6, P7)
  ]

In [7]:
# shutil.rmtree('/app/_data/ScaledYOLOv4/runs/exp0_yolov4lp7_1')

In [8]:
!python train.py --img 1024 --batch 6 --epochs 150 --data /app/_data/ScaledYOLOv4/data/DataFile/data.yaml --cfg /app/_data/ScaledYOLOv4/data/DataFile/customYOLOv4l.yaml --weights /app/_data/ScaledYOLOv4/weights/yolov4-p7.pt --name yolov4lp7_1 


Using CUDA device0 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24268MB)

Namespace(adam=False, batch_size=6, bucket='', cache_images=False, cfg='/app/_data/ScaledYOLOv4/data/DataFile/customYOLOv4l.yaml', data='/app/_data/ScaledYOLOv4/data/DataFile/data.yaml', device='', epochs=150, evolve=False, global_rank=-1, hyp='data/hyp.finetune.yaml', img_size=[1024, 1024], local_rank=-1, logdir='runs/', multi_scale=False, name='yolov4lp7_1', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, sync_bn=False, total_batch_size=6, weights='/app/_data/ScaledYOLOv4/weights/yolov4-p7.pt', world_size=1)
Start Tensorboard with "tensorboard --logdir runs/", view at http://localhost:6006/
Hyperparameters {'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate'